<a href="https://colab.research.google.com/github/erwanBellon/Project-ML-SDM/blob/main/code/optimised_ANN_model_habitat_amount_3000_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I: Setup



In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Cloning repo or fetch latest changes and path management
%cd /content
!rm -rf Project-ML-SDM
!git clone https://github.com/erwanBellon/Project-ML-SDM.git
%cd /content/Project-ML-SDM
!git pull

import os
from pathlib import Path

# Move into the project directory
%cd /content/Project-ML-SDM/code
print("Current working directory:", Path.cwd())

# Define main project dir and outputs
PROJECT_ROOT_DIR = Path.cwd().parent
OUTPUTS_PATH = PROJECT_ROOT_DIR / "outputs"
OUTPUTS_PATH.mkdir(parents=True, exist_ok=True)
print("Outputs will be saved to:", OUTPUTS_PATH)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
from tensorflow.keras import regularizers
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU detected. CNNs can be slow without GPU.")

# Common imports
import pandas as pd
import numpy as np
!pip install rasterio
import rasterio
!pip install -U keras-tuner
import keras_tuner as kt
!pip install pyreadr
import pyreadr
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


# To make notebook reproducible
np.random.seed(123)
tf.random.set_seed(123)

# For plots
import matplotlib.pyplot as plt
%matplotlib inline

# Load Tensorboard
%load_ext tensorboard

/content
Cloning into 'Project-ML-SDM'...
remote: Enumerating objects: 545, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 545 (delta 54), reused 19 (delta 19), pack-reused 450 (from 1)
Receiving objects: 100% (545/545), 17.62 MiB | 10.54 MiB/s, done.
Resolving deltas: 100% (405/405), done.
/content/Project-ML-SDM
Already up to date.
/content/Project-ML-SDM/code
Current working directory: /content/Project-ML-SDM/code
Outputs will be saved to: /content/Project-ML-SDM/outputs
No GPU detected. CNNs can be slow without GPU.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.2/776.2 kB 10.6 MB/s eta 0:00:00


In [ ]:
print(Path.cwd())

/content/Project-ML-SDM/code


# Part 2: Load files
## 2.1: Load the presences and absences landcover crops

In [ ]:
# --- Load table data ---
rds_path = Path("../data/Table_preds/function_3_100.rds")
result = pyreadr.read_r(rds_path)
table_df = result[None]

# Select relevant columns
feature_df = table_df[['MAP','MAT','habitat_amount_3000']].astype(float)

# Labels come from presence/absence index lists
# Load image index files (no images) (just to get the indices that I need from my datatable)
presences_path = Path("../data/cropped_landcover/presences")
absences_path = Path("../data/cropped_landcover/absences")

def extract_indices(folder):
    tif_files = sorted(folder.glob("*.tif"))
    indices = []
    for tif in tif_files:
        idx = int(tif.stem.split("_")[-1])
        indices.append(idx)
    return np.array(indices)

indices_pres = extract_indices(presences_path)
indices_abs = extract_indices(absences_path)

# Build label vector
y = np.concatenate([
    np.ones(len(indices_pres)),
    np.zeros(len(indices_abs))
]).astype(np.float32)

# Extract corresponding rows
feature_all = feature_df
features = feature_all.iloc[np.concatenate([indices_pres, indices_abs])]
features = features.reset_index(drop=True)

# Normalize: no need to normalize after to avoid dataleakage
#table_features = (table_features - table_features.min()) / (table_features.max() - table_features.min())
#table_features = table_features.to_numpy(dtype=np.float32)

# Train/valid/test split
X_train, X_temp, y_train, y_temp = train_test_split(
    features, y, test_size=0.2, random_state=123, stratify=y
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=123, stratify=y_temp
)


# Normalize using TRAIN ONLY

min_vals = X_train.min(axis=0)
max_vals = X_train.max(axis=0)

X_train = (X_train - min_vals) / (max_vals - min_vals)
X_valid = (X_valid - min_vals) / (max_vals - min_vals)
X_test  = (X_test  - min_vals) / (max_vals - min_vals)





Note that there is some data leakage here as I pre-process part of my test set with the data of the training for normalizing the features. I don't have the time to change that... I've realised to late

## 2.2 Preprocess the data

In [ ]:
BATCH_SIZE = 8

def preprocess_table(table, label):
    table = tf.cast(table, tf.float32)
    label = tf.cast(label, tf.float32)
    return table, label

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
      .shuffle(1000)
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

valid_ds = (
    tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
      .map(preprocess_table)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

train_ds.name = "Training"
valid_ds.name = "Validation"
test_ds.name = "Test"


# 3. Optimised ANN model Training

Doing an optised ANN using hyperparameter hyperband tuning

In [ ]:
def build_model(hp):

    # ---- Hyperparameters to tune ----
    units_1 = hp.Choice("units_1", values=[8, 16, 32])
    units_2 = hp.Choice("units_2", values=[4, 8, 16])
    units_3 = hp.Choice("units_3", values=[16, 32, 64])

    dropout_rate = hp.Choice("dropout", values=[0.1, 0.2, 0.3])
    l2_strength = hp.Choice("l2", values=[1e-4, 1e-3, 1e-2])
    lr = hp.Choice("lr", values=[1e-3, 5e-4, 1e-4])

    # ---- Model ----
    table_input = keras.Input(shape=(3,), name="table_input")

    y = layers.Dense(
        units_1,# 16
        activation="relu",
        kernel_regularizer=regularizers.l2(l2_strength)# 0.001
    )(table_input)

    y = layers.Dense(
        units_2,# 16
        activation="relu",
        kernel_regularizer=regularizers.l2(l2_strength)# 0.001
    )(y)

    z = layers.Dense(units_3, activation="relu")(y) #64
    z = layers.Dropout(dropout_rate)(z)# 0.3
    z = layers.Dense(8, activation="relu")(z)

    output = layers.Dense(1, activation="sigmoid", name="suitability")(z)

    model = keras.Model(table_input, output)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc")]
    )

    return model

Create the hyperband tuner

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective=kt.Objective("val_auc", direction="max"),
    max_epochs=50,
    factor=3,
    directory="hyper_param_tuning",
    project_name="ANN_table_tuning",
    overwrite=True   # start fresh
)


In [ ]:
stop_early = keras.callbacks.EarlyStopping(
    monitor="val_auc",
    patience=10,
    mode="max",
    restore_best_weights=True
)


tuner.search(
    train_ds,
    validation_data=valid_ds,
    epochs=50,
    callbacks=[stop_early]
)


Trial 90 Complete [00h 00m 09s]
val_auc: 0.8104166984558105

Best val_auc So Far: 0.8812500238418579
Total elapsed time: 00h 09m 12s


In [ ]:
best_hp = tuner.get_best_hyperparameters(1)[0]

print("Best hyperparameters:")
for k, v in best_hp.values.items():
    print(f"  {k}: {v}")

best_model = tuner.get_best_models(1)[0]


Best hyperparameters:
  units_1: 16
  units_2: 16
  units_3: 64
  dropout: 0.3
  l2: 0.0001
  lr: 0.001
  tuner/epochs: 50
  tuner/initial_epoch: 17
  tuner/bracket: 2
  tuner/round: 2
  tuner/trial_id: 0067


In [ ]:
history = best_model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=50,
    callbacks=[stop_early]
)


Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - auc: 0.7181 - loss: 0.6065 - val_auc: 0.8667 - val_loss: 0.5278
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - auc: 0.7556 - loss: 0.5725 - val_auc: 0.8750 - val_loss: 0.5261
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - auc: 0.7904 - loss: 0.5372 - val_auc: 0.8729 - val_loss: 0.5175
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - auc: 0.7637 - loss: 0.5582 - val_auc: 0.8792 - val_loss: 0.5229
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - auc: 0.8180 - loss: 0.5265 - val_auc: 0.8771 - val_loss: 0.5173
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - auc: 0.8023 - loss: 0.5596 - val_auc: 0.8792 - val_loss: 0.5201
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - auc: 0.7487 - loss: 0.5136 - val_auc: 0.8833 - val_loss: 0.5154
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - auc: 0.7541 - loss: 0.5722 - val_auc: 0.8771 - val_loss: 0.5187
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - auc: 0.7690 - loss:

# 4. Model evaluation

In [ ]:
test_loss, test_auc = best_model.evaluate(test_ds)
print("\n Final Test AUC =", test_auc)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - auc: 0.8938 - loss: 0.5421 

 Final Test AUC = 0.8583333492279053


MaxTSS is not implemented in keras. I'll compute
it there:

In [ ]:
def compute_maxTSS(y_true, y_prob):
    thresholds = np.linspace(0, 1, 200)
    best_tss = -2
    best_threshold = None

    for t in thresholds:
        preds = (y_prob >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()

        sensitivity = tp / (tp + fn + 1e-9)
        specificity = tn / (tn + fp + 1e-9)
        tss = sensitivity + specificity - 1

        if tss > best_tss:
            best_tss = tss
            best_threshold = t

    return best_tss, best_threshold

In [ ]:
y_prob = best_model.predict(test_ds)
y_true = np.concatenate([y for (x, y) in test_ds])
max_tss, best_threshold = compute_maxTSS(y_true, y_prob)

print("MaxTSS:", max_tss)
print("Optimal threshold:", best_threshold)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
MaxTSS: 0.5999999998950001
Optimal threshold: 0.3969849246231156


Save my best model and mannually load it in GitHub

In [ ]:
from google.colab import files
best_model.save("Optimised_ANN_HA_3000_bestModel.keras")
files.download("Optimised_ANN_HA_3000_bestModel.keras")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>